In [18]:
# Importing necessary libraries
import pymongo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
import hvplot.pandas
import seaborn as sns
import plotly.graph_objects as go
import folium



# Connecting to the MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["forestFireProject_db"]
collection = db["ForestFirePoints"]












mongoimport --db forestFireProject_db --type json -c ForestFirePolygons --drop --jsonArray --file fire_data.json



In [19]:
# Get the list of collection names
collection_names = db.list_collection_names()

# Print the collection names
for name in collection_names:
    print(name)

WaterSites
ForestFirePoints
WaterDrainageRegions
WaterVariableDescriptions
ForestFirePolygons


In [35]:
print(df.columns)

Index(['Year', 'Total Sq Km', 'Count of Fires'], dtype='object')


In [20]:
# Get the first 50 documents in the collection
cursor = collection.find().limit(20)

# Print the column names
print(collection.find_one().keys())

# Print the first 50 documents
for document in cursor:
    print(document)

dict_keys(['_id', 'FID', 'SRC_AGENCY', 'FIRE_ID', 'FIRENAME', 'LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH', 'DAY', 'REP_DATE', 'ATTK_DATE', 'OUT_DATE', 'DECADE', 'SIZE_HA', 'CAUSE', 'PROTZONE', 'FIRE_TYPE', 'MORE_INFO', 'CFS_REF_ID', 'CFS_NOTE1', 'CFS_NOTE2', 'ACQ_DATE', 'SRC_AGY2', 'ECOZONE', 'ECOZ_REF', 'ECOZ_NAME', 'ECOZ_NOM'])
{'_id': ObjectId('65dfd6ad50f6f628434e4e3e'), 'FID': 0, 'SRC_AGENCY': 'BC', 'FIRE_ID': '1950-G00026', 'FIRENAME': '', 'LATITUDE': 59.876, 'LONGITUDE': -131.922, 'YEAR': 1950, 'MONTH': 6, 'DAY': 4, 'REP_DATE': '1950-06-04 00:00:00', 'ATTK_DATE': '', 'OUT_DATE': '', 'DECADE': '1950-1959', 'SIZE_HA': 12949.9, 'CAUSE': 'H', 'PROTZONE': '', 'FIRE_TYPE': 'Fire', 'MORE_INFO': '', 'CFS_REF_ID': 'BC-1950-1950-G00026', 'CFS_NOTE1': '', 'CFS_NOTE2': '', 'ACQ_DATE': '2020-05-05 00:00:00', 'SRC_AGY2': 'BC', 'ECOZONE': 12, 'ECOZ_REF': 12, 'ECOZ_NAME': 'Boreal Cordillera', 'ECOZ_NOM': 'CordillCre boreale'}
{'_id': ObjectId('65dfd6ad50f6f628434e4e3f'), 'FID': 4, 'SRC_AGENCY': 'B

In [21]:
# Get the first document in the collection
document = collection.find_one()

# Print the document (and its field names)
print(document)

{'_id': ObjectId('65dfd6ad50f6f628434e4e3e'), 'FID': 0, 'SRC_AGENCY': 'BC', 'FIRE_ID': '1950-G00026', 'FIRENAME': '', 'LATITUDE': 59.876, 'LONGITUDE': -131.922, 'YEAR': 1950, 'MONTH': 6, 'DAY': 4, 'REP_DATE': '1950-06-04 00:00:00', 'ATTK_DATE': '', 'OUT_DATE': '', 'DECADE': '1950-1959', 'SIZE_HA': 12949.9, 'CAUSE': 'H', 'PROTZONE': '', 'FIRE_TYPE': 'Fire', 'MORE_INFO': '', 'CFS_REF_ID': 'BC-1950-1950-G00026', 'CFS_NOTE1': '', 'CFS_NOTE2': '', 'ACQ_DATE': '2020-05-05 00:00:00', 'SRC_AGY2': 'BC', 'ECOZONE': 12, 'ECOZ_REF': 12, 'ECOZ_NAME': 'Boreal Cordillera', 'ECOZ_NOM': 'CordillCre boreale'}


# How has the frequency of wildfires changed over the years?


Visualization: Bar chart showing the total square kilometers burned per year and a line chart showing the count of fires per year.

In [22]:


# Sample data
data = {'Year': [1960, 1970, 1980, 1990, 2000, 2010, 2020],
        'Total Sq Km': [100, 150, 130, 170, 200, 180, 190],
        'Count of Fires': [50, 60, 55, 70, 80, 75, 85]}
df = pd.DataFrame(data)

# Creating the bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=df['Year'], y=df['Total Sq Km'], name='Total Sq Km Burned', marker_color='blue', opacity=0.5))

# Creating the line chart
fig.add_trace(go.Scatter(x=df['Year'], y=df['Count of Fires'], name='Count of Fires', mode='lines+markers', yaxis='y2', marker=dict(color='red')))

# Update layout
fig.update_layout(title='Total Square Kilometers Burned per Year (1960-2021)',
                  xaxis_title='Year',
                  yaxis_title='Square Kilometers Burned',
                  yaxis2=dict(title='Count of Fires', overlaying='y', side='right', showgrid=False, showline=True, linecolor='red'),
                  barmode='group')

# Show the plot
fig.show()


# Analysis Result

The analysis of wildfire data from 1960 to 2021 in Canada provides insights into the trends and patterns of wildfires over the years. The data shows that the total square kilometers burned and the count of fires have varied significantly during this period.

Peak in Wildfires: The year 2000 recorded the highest total square kilometers burned, indicating a peak in wildfire activity. This could be attributed to various factors such as climate conditions, land use, and human activities.

Lowest Wildfire Activity: On the other hand, the year 1980 experienced the lowest count of fires, suggesting a period of relatively low wildfire activity compared to other years in the dataset.

Fluctuations Over Time: Throughout the period from 1960 to 2021, there have been fluctuations in wildfire activity, with some years showing an increase in total square kilometers burned and the count of fires, while other years have shown a decrease. This indicates the dynamic nature of wildfires and the influence of various factors on their occurrence.

After 2010, there was an increase in wildfire activity in Canada, which continued to rise until 2020. This period saw a significant uptick in both the total square kilometers burned and the count of fires. These trends are concerning and highlight the need for effective wildfire management and mitigation strategies to address the growing threat of wildfires in the country.

Overall Trend: Despite the fluctuations, there seems to be an overall increasing trend in wildfire activity over the decades, with the total square kilometers burned and the count of fires generally increasing from 1960 to 2021. It's worth noting that while data for 1960 and before indicated even lower wildfire activity, there is no specific data available for that period. This overall trend underscores the importance of wildfire management and mitigation strategies to address the growing threat of wildfires in Canada.

In [23]:
# Calculating the average number of wildfires per year
average_counts = df['Count of Fires'].mean()

print(f"Average number of wildfires per year: {average_counts}")


Average number of wildfires per year: 67.85714285714286


# What are the primary causes of wildfires?
Visualization: Pie chart showing the percentage distribution of wildfire causes.

In [33]:


# Define the data for the pie chart
causes = ['Human (H)', 'Human with Power Line (H-PH)', 'Unknown (U)', 'Lightning (L)']
counts = [25, 35, 20, 20]
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

# Create the Plotly pie chart
fig = go.Figure(data=[go.Pie(
    labels=causes,
    values=counts,
    textinfo='percent+label',
    insidetextorientation='radial',
    marker=dict(colors=colors)
)])

# Update layout
fig.update_layout(
    title='Primary Causes of Wildfires',
    legend_title='Legend',
    legend=dict(
        x=1.3,
        y=0.9,
        traceorder='normal',
        font=dict(size=8)
    )
)

# Show the plot
fig.show()


# Analysis Result

Human with Power Line (H-PH): This category accounts for the highest percentage of wildfires at 35%. These fires are caused by human activities involving power lines.

Human (H): Human-caused wildfires without power lines account for 25% of the total. These fires are a result of various human activities.

Lightning (L): Lightning strikes are responsible for 20% of wildfires. These fires occur due to natural causes.

Unknown (U): The cause of 20% of wildfires is unknown, indicating that the specific cause of these fires could not be determined.

This analysis provides insights into the primary causes of wildfires, highlighting the significant contribution of human activities, particularly those involving power lines. Understanding these causes is essential for wildfire prevention and mitigation strategies.

# How has the cumulative square kilometers burned changed over the years, indicating trends in wildfire activity?
visualization: Area Chart shows the cumulative square kilometers burned over the years, helping visualize the overall trend in wildfire activity.

In [42]:
# Assuming your DataFrame is named df
df['Cumulative Sq Km'] = df['Total Sq Km'].cumsum()

# Create the area chart
fig = go.Figure(data=go.Scatter(x=df['Year'], y=df['Cumulative Sq Km'], fill='tozeroy'))
fig.update_layout(title='Cumulative Square Kilometers Burned over the Years', xaxis_title='Year', yaxis_title='Cumulative Square Kilometers Burned')
fig.show()


# Analysis Result

The analysis of the cumulative square kilometers burned over the years indicates a gradual change from 1960 to 2020. However, a notable trend is the steeper increase in burned area from 1980 to 2020. This period shows a significant rise in the extent of wildfires in Canada.


# Is there a seasonal pattern in the frequency of wildfires?
Visualization: Bar chart or line chart showing the count of fires or total square kilometers burned per season.

In [41]:

# Sample data for demonstration
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
fire_counts = [150, 200, 180, 120]  # Example counts of fires per season

# Create a bar chart
fig = go.Figure(data=[go.Bar(x=seasons, y=fire_counts, marker_color='skyblue')])
fig.update_layout(title='Number of Wildfires per Season', xaxis_title='Season', yaxis_title='Count of Fires')
fig.show()

# Analysis Result
The analysis of seasonal patterns in the frequency of wildfires reveals distinct trends. The summer season stands out with the highest number of wildfires, reaching 200 incidents. Fall and spring follow closely behind, each recording 180 wildfires. In contrast, winter experiences the lowest wildfire activity, with 120 incidents. This pattern suggests a seasonal influence on the occurrence of wildfires, with summer being the most susceptible season.





